In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

data = load_boston()
data.keys()

dict_keys(['data', 'target', 'feature_names', 'DESCR', 'filename'])

In [2]:
data["feature_names"]

array(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD',
       'TAX', 'PTRATIO', 'B', 'LSTAT'], dtype='<U7')

In [3]:
X, y = data["data"], data["target"]

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import VotingRegressor


X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

rfc_reg = RandomForestRegressor(max_depth=3, criterion='mse').fit(X_train, y_train)
print(f'Random Forest r2_score: {rfc_reg.score(X_val, y_val)}')

dt_reg = DecisionTreeRegressor(max_depth=4).fit(X_train, y_train)
print(f'Decision Tree r2_score: {dt_reg.score(X_val, y_val)}')

knn_reg = KNeighborsRegressor(n_neighbors=5).fit(X_train, y_train)
print(f'KNN r2_score: {knn_reg.score(X_val, y_val)}')

ridge_reg = Ridge().fit(X_train, y_train)
print(f'Ridge r2_score: {ridge_reg.score(X_val, y_val)}')

#sklearn creates a fresh copy using clone internally
vote_reg = VotingRegressor([('rfc_reg',rfc_reg),
                            ('dt_reg',dt_reg),
                            ('knn_reg',knn_reg),
                            ('ridge_reg',ridge_reg)
                           ],
                          weights=None,
                          n_jobs=2).fit(X_train, y_train)
print(f'Vote regresion r2_score: {vote_reg.score(X_val, y_val)}')

modelErrors = {'true':y_val,
               'rfc':rfc_reg.predict(X_val),
               'dt':dt_reg.predict(X_val),
               'knn':knn_reg.predict(X_val),
               'ridge':ridge_reg.predict(X_val)
              }

errors = {}
for name in modelErrors.keys():
    if name == 'true':
        continue
    errors[name+'_error'] =  modelErrors['true']-modelErrors[name]

dfE = pd.DataFrame(errors)
dfE.corr().style.background_gradient(cmap="summer",low=2)

Random Forest r2_score: 0.8137206925454441
Decision Tree r2_score: 0.8046111865480169
KNN r2_score: 0.6315333003650656
Ridge r2_score: 0.7539964591177476
Vote regresion r2_score: 0.8535426368221811


In [6]:
fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
pd.plotting.scatter_matrix(dfE,ax=ax,alpha=0.5)
plt.show()

/Users/maxschallwig/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: To output multiple subplots, the figure containing the passed axes is being cleared
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
# try removing rfc or dt
vote_reg_red = VotingRegressor([('rfc_reg',rfc_reg),
                                #('dt_reg',dt_reg),
                                ('knn_reg',knn_reg),
                                ('ridge_reg',ridge_reg)
                               ],
                               weights=None,
                               n_jobs=2).fit(X_train, y_train)
print(f'Smaller Vote regresion r2_score: {vote_reg_red.score(X_val, y_val)}')

Smaller Vote regresion r2_score: 0.8385530912595762
